### Import Libraries

In [1]:
from nsepython import *
import pandas as pd
import yfinance as yf
import numpy as np
import math
from scipy.stats import iqr # For Finding Volume Outliers

import plotly.graph_objects as go
from plotly.subplots import make_subplots

Read the CSV file

In [2]:
## Stock Symbols contains list of 1250 NSE stock symbols (Need to update it every quarter / year)

stock_data = pd.read_csv('stock_data.csv')
stock_data.set_index('SYMBOL', inplace=True)
stock_data = stock_data[stock_data['MARKET_CAP'] < 50000]
stock_data = stock_data[stock_data['MARKET_CAP'] > 2000]
len(stock_data)

799

In [3]:
df = pd.DataFrame({'SYMBOL':[], 'MOMENTUM':[], 'CONSISTENCY': [], 'CONFIRMATION': [], 'HPF':[], 'HPM':[]})

t13 = 7 * 30
t7 = 3 * 30
t1 = 0.3 * 30

for symbol in stock_data.index.values:
    start_date = pd.to_datetime('today') + pd.DateOffset(-t13)
    end_date = pd.to_datetime('today') + pd.DateOffset(-t1)
    
    data = yf.download(symbol + ".NS", start=start_date, end=end_date, progress=False)
    
    cut_off_date = pd.to_datetime('today') + pd.DateOffset(-10) # To exclude new stocks which don't have data
    
    if(data.shape[0]==0 or (data.index.min() > cut_off_date)):
        continue
        
    t_7 = pd.to_datetime('today') + pd.DateOffset(-t7)
    t_1 = pd.to_datetime('today') + pd.DateOffset(-t1)
    
    data['Return'] = (data['Close'] - data['Close'].shift(1)) / data['Close'].shift(1)
    sub_data = data[(data.index > t_7) & (data.index < t_1)]
    
    no_of_rows = data.shape[0]
    no_of_rows_sub_data = sub_data.shape[0]
    
    ## INDICATORS

    # Momentum: Historical cumulative return from t-13 to t-2 included
    
    momentum = (data['Close'][no_of_rows-1] - data['Close'][0]) / data['Close'][0]
    
    # Consistency: Number of days with positive returns (historical from t-7 to t-2 included)
    
    consistency = sub_data[sub_data['Return'] > 0].shape[0]
    
    # Confirmation Sum of daily returns (historical from t-7 to t-2 included)
    
    confirmation = sub_data['Return'].sum()
    
    # Highest Price Factor - Current price/Highest price reached in historical t-7 to t-2
    
    highest_price_factor = sub_data['Close'][no_of_rows_sub_data-1] / sub_data['Close'].max()
    
    # High-to-Price Momentum: Log ratio of the highest price reached to the initial price during the 
    # formation period from t-13 to t-2
    
    high_to_momentum = math.log(data['Close'].max() / data['Close'][0])
    
    # Volatility Indicator
    
    volatility = 1 - sub_data['Return'].std()
    
    
    new_row = pd.DataFrame({'SYMBOL':[symbol], 
                              'MOMENTUM':[momentum], 
                              'CONSISTENCY': [consistency], 
                              'CONFIRMATION': [confirmation], 
                              'HPF':[highest_price_factor],
                              'VOLATILITY':[volatility], 
                              'HPM':[high_to_momentum]})

    df = pd.concat([df, new_row], ignore_index=True)

df.set_index('SYMBOL', inplace=True)
df_rank = df.rank()

df_rank['MACRO_SIGNAL'] = df_rank['MOMENTUM']
df_rank['PERSISTENCY_SIGNAL'] = (df_rank['CONSISTENCY'] + df_rank['CONFIRMATION'] + df_rank['HPF'] + df['VOLATILITY']) / 4
df_rank['MOMENTUM_SIGNAL'] = 0.5 * df_rank['MACRO_SIGNAL'] + 0.5 * df_rank['PERSISTENCY_SIGNAL']
df_rank['MOMENTUM_HTP_SIGNAL'] = 0.5 * df_rank['MOMENTUM_SIGNAL'] + 0.5 * df_rank['HPM']

result = df_rank['MOMENTUM_HTP_SIGNAL'].sort_values(ascending=False)
# stock_data['MOMENTUM_HTP_SIGNAL'] = 
print('Execution Completed !!!')


1 Failed download:
['TATACOFFEE.NS']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-07-01 16:22:44.871541 -> 2024-01-18 16:22:44.871647)')


Execution Completed !!!


In [4]:
momentum = stock_data[stock_data.index.isin(result.index[50:100])]
momentum.head(40)

,MARKET_CAP,INDUSTRY,SECTOR,MACRO,FNO
SYMBOL,,,,,
RVNL,37812.0,Construction,Construction,Industrials,0.0
NLCINDIA,35013.0,Power,Power,Utilities,0.0
NIACL,34550.0,Insurance,Financial Services,Financial Services,0.0
ANGELONE,29296.0,Capital Markets,Financial Services,Financial Services,0.0
IREDA,27576.0,Finance,Financial Services,Financial Services,0.0
HINDCOPPER,26289.0,Non - Ferrous Metals,Metals & Mining,Commodities,1.0
TATAINVEST,21627.0,Finance,Financial Services,Financial Services,0.0
BRIGADE,20738.0,Realty,Realty,Consumer Discretionary,0.0
CESC,17504.0,Power,Power,Utilities,0.0


In [6]:
try:
    figure_html = open('sif.html', 'w')

    for symbol in result.index.values[32:150]:
        
        try:
            if(symbol not in stock_data.index):
                print(f"{symbol} is not part of our stock data!")
                continue
            data = nse_past_results(symbol)

            df = pd.DataFrame(data.get('resCmpData'))
            isBank = data.get('bankNonBnking') == 'B'
    
            if(not isBank):
                if len(df.columns) == 0:
                    print(f"{symbol} no past results found for this symbol")
                    continue

                new_columns = {
                    're_to_dt': 'DATE',
                    're_create_dt': 'RELEASE_DATE',
                    're_net_sale': 'INCOME',
                    're_oth_tot_exp': 'COST',
                    're_oth_inc_new': 'OTHER_INCOME',
                    're_pro_loss_bef_tax': 'PBT',
                    're_net_profit': 'PROFIT',
                    're_basic_eps_for_cont_dic_opr': 'EPS'
                }

                df.rename(columns=new_columns, inplace=True)

                df = df[['DATE', 'RELEASE_DATE', 'INCOME', 'COST', 'OTHER_INCOME', 'PBT', 'PROFIT', 'EPS']]
                df.set_index('DATE', inplace=True)

                for col in df.columns:
                    if col != 'RELEASE_DATE':
                        df[col] = pd.to_numeric(df[col])

                for col in df.columns:
                    if col not in ['EPS', 'RELEASE_DATE']:
                        df[col] = round(df[col] / 100)
                        df[col] = df[col].round(0).astype('Int64')

                df['OP_INCOME'] = df['INCOME'] - df['COST']
                df['OPM'] = (100 * (df['INCOME'] - df['COST']) / df['INCOME']).round(2).astype(str) + "%"
                df['TAX'] = (100 * (df['PBT'] - df['PROFIT']) / df['PBT']).round(2).astype(str) + "%"

                df = df[['INCOME', 'OP_INCOME', 'OPM', 'OTHER_INCOME', 'TAX', 'PROFIT', 'EPS', 'RELEASE_DATE']]

                df = df.astype(str)
                df = df.transpose()
                df = df.iloc[:, ::-1]
            else:
                new_columns = {'re_to_dt': 'DATE', 
                   're_create_dt': 'RELEASE_DATE', 
                   're_int_earned':'NET_INTEREST_INCOME',
                   're_tot_exp_exc_pro_cont':'OPERATING_EXPENSE', 
                   're_oth_inc':'OTHER_INCOME',
                   're_oth_pro_cont':'PROVISION',
                   're_pro_loss_bef_tax':'PBT', 
                   're_con_pro_loss':'PROFIT', 
                   're_basic_eps':'EPS'}
    
                df.rename(columns=new_columns, inplace=True)
    
                df = df[['DATE', 'RELEASE_DATE', 'NET_INTEREST_INCOME', 'OPERATING_EXPENSE', 'PROVISION', 
                         'OTHER_INCOME','PBT', 'PROFIT', 'EPS']]
                df.set_index('DATE', inplace=True)
    
                for col in df.columns:
                    if col != 'RELEASE_DATE':
                        df[col] = pd.to_numeric(df[col])
    
                for col in df.columns:
                    if col not in ['EPS','RELEASE_DATE']:
                        df[col] = round(df[col] / 100)
                        df[col] = df[col].round(0).astype('Int64')
            
                df['OP_INCOME'] = df['NET_INTEREST_INCOME'] - df['OPERATING_EXPENSE']
                df['OPM'] = (100 * (df['NET_INTEREST_INCOME'] - df['OPERATING_EXPENSE']) / df['NET_INTEREST_INCOME']).round(2).astype(str) + "%"
                df['TAX'] = (100 * (df['PBT'] - df['PROFIT']) / df['PBT']).round(2).astype(str) + "%"
    
                df = df[['NET_INTEREST_INCOME', 'OP_INCOME', 'OPM','OTHER_INCOME', 'PROVISION', 'TAX', 'PROFIT', 'EPS', 'RELEASE_DATE']]
    
                df = df.astype(str)
                df = df.transpose()
                df = df.iloc[:, ::-1]
        
    
            ### Ploting 
    
            data = yf.download(symbol + ".NS", start='2023-01-01', end=pd.to_datetime('today')+pd.DateOffset(1), progress=False)
    
            data['20DMA'] = data['Close'].rolling(window=20).mean()
            data['50DMA']= data['Close'].rolling(window=50).mean()
            data['100DMA']= data['Close'].rolling(window=100).mean()
    
            data['diff'] = data['Close'] - data['Open']
            data.loc[data['diff'] >= 0, 'color'] = 'green'
            data.loc[data['diff'] < 0, 'color'] = 'red'
    
            plot_data = data[data.index > (pd.to_datetime('today') - pd.DateOffset(days=270))]

            figure = make_subplots(specs=[[{"secondary_y": True}]])
    
            # Subplot 1 - Price Candlestick
    
            figure.add_trace(go.Candlestick(x = plot_data.index,
                              open = plot_data['Open'],
                              high = plot_data['High'],
                              low = plot_data['Low'],
                              close = plot_data['Close'],
                              name='Price'))
            figure.update_yaxes(range=[plot_data['Close'].min()*0.9, plot_data['Close'].max()*1.05])
            figure.update_xaxes(rangebreaks = [ dict(bounds=['sat','mon']) ] ) # hide weekends 
            figure.update_layout(title={'text':symbol, 'x':0.5})
            figure.update_layout(xaxis_rangeslider_visible=False)  #hide range slider
    
            # Subplot 2 - Volume and DMAs
    
            figure.add_trace(go.Scatter(x=plot_data.index, y=plot_data['20DMA'], marker_color='blue',name='20 Day MA'))
            figure.add_trace(go.Scatter(x=plot_data.index, y=plot_data['50DMA'], marker_color='orange',name='50 Day MA'))
            figure.add_trace(go.Scatter(x=plot_data.index, y=plot_data['100DMA'], marker_color='green',name='100 Day MA'))
    
            figure.add_trace(go.Bar(x=plot_data.index, y=plot_data['Volume'], name='Volume', marker={'color':plot_data['color']}),secondary_y=True)
            figure.update_yaxes(range=[0, plot_data['Volume'].max()*5], secondary_y=True)
            figure.update_yaxes(visible=False, secondary_y=True)
    
            # Adding Text Below
            stock_row = stock_data.loc[symbol]
            figure.add_annotation(dict(font=dict(color='black', size=16.5),
                x=0.5,  # Center aligned horizontally
                y=-0.12, showarrow=False,
                text="SECTOR - " + stock_row['SECTOR']+"  MARKET CAP - "+str(stock_row['MARKET_CAP'].astype(int)) + "Cr",
                textangle=0,
                xanchor='center',  # Center aligned horizontally
                yanchor='bottom',  # Aligned to the bottom
                xref="paper", yref="paper"))
    
            # Adding Earnings Date in the figure
            #     figure.add_annotation(go.layout.Annotation(text="Q3",
            #         x=pd.to_datetime(df.loc['RELEASE_DATE']['31-DEC-2023']),
            #         y=plot_data['Close'].min()*0.9, 
            #         showarrow=True, arrowhead=2, arrowsize=1, arrowwidth=3, arrowcolor="yellow",
            #         ax=0, ay=-40))
    
            figure.add_annotation(go.layout.Annotation(text="Q2",
                x=pd.to_datetime(df.loc['RELEASE_DATE']['30-SEP-2023']),
                y=plot_data['Close'].min()*0.9, 
                showarrow=True, arrowhead=2, arrowsize=1, arrowwidth=3, arrowcolor="yellow",
                ax=0, ay=-40))
    
    
            figure.add_annotation(go.layout.Annotation(text="Q1",
                x=pd.to_datetime(df.loc['RELEASE_DATE']['30-JUN-2023']),
                y=plot_data['Close'].min()*0.9, 
                showarrow=True, arrowhead=2, arrowsize=1, arrowwidth=3, arrowcolor="yellow",
                ax=0, ay=-40))
    
    
            html_table = df.style.set_table_styles([
            {'selector': 'thead th', 'props': [('background-color', 'lightblue'), ('color', 'black'), ('text-align', 'center'), ('font-weight', 'bold')]},
            {'selector': 'tbody td', 'props': [('text-align', 'center')]}]).render()
    
            ### Writing the results dataframe and plot to the html page
            figure_html.write(html_table)
            figure_html.write(figure.to_html(full_html=False))
        
        except Exception as symbol_exception:
            print(f"Error processing symbol {symbol}: {symbol_exception}")
            continue
except Exception as main_exception:
    print(f"Main error: {main_exception}")

finally:
    # Close the HTML file
    figure_html.close()

print('Execution Completed!!!!')


1 Failed download:
['HUDCO.NS']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2023-01-01 -> 2024-01-28 17:19:29.206023)')


Error processing symbol HUDCO: cannot set a frame with no defined index and a scalar


/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_5226/2678161537.py:164: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_5226/2678161537.py:164: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_5226/2678161537.py:164: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_5226/2678161537.py:164: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_5226/2678161537.py:164: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_5226/2678161537.py:164: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipy

Error processing symbol MOTILALOFS: '30-JUN-2023'


/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_5226/2678161537.py:164: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_5226/2678161537.py:164: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_5226/2678161537.py:164: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`



...

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_5226/2678161537.py:164: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_5226/2678161537.py:164: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_5226/2678161537.py:164: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_5226/2678161537.py:164: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_5226/2678161537.py:164: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_5226/2678161537.py:164: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipy

Error processing symbol SANDUMA: '30-JUN-2023'


/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_5226/2678161537.py:164: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_5226/2678161537.py:164: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_5226/2678161537.py:164: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_5226/2678161537.py:164: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_5226/2678161537.py:164: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_5226/2678161537.py:164: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`

/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipy

Execution Completed!!!!


/var/folders/y8/cj_f7jps7ylfv21xfzjbynm40000gn/T/ipykernel_5226/2678161537.py:164: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`



In [6]:
data = yf.download('IRCTC' + ".NS", start='2023-01-01', end='2023-12-31', progress=False)

In [7]:
data['Return'] = (data['Close'] - data['Close'].shift(1)) / data['Close'].shift(1)

In [8]:
data['Return'].std()

0.016741716048074867